In [1]:
!pip install transformers

     |████████████████████████████████| 2.8 MB 8.3 MB/s 
     |████████████████████████████████| 636 kB 67.8 MB/s 
     |████████████████████████████████| 52 kB 1.9 MB/s 
     |████████████████████████████████| 3.3 MB 59.9 MB/s 
     |████████████████████████████████| 895 kB 63.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
import pandas as pd
import numpy as np
import torch
from transformers import BertTokenizer

In [3]:
train_data=pd.read_csv("/content/drive/MyDrive/train_qa.csv")

In [4]:
test_data=pd.read_csv("/content/drive/MyDrive/test_qa.csv")

In [5]:
# torch imports
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

In [6]:
batch_size=8

Keeping the text data for passing to BART encoder


In [7]:
doctrain_data=[t for t in train_data['Z']]
doctest_data=[t for t in test_data['Z']]

In [8]:
qtrain_data=[t for t in train_data['X']]
qtest_data=[t for t in test_data['X']]

In [9]:
ytrain_data=[t for t in train_data['Y']]
ytest_data=[t for t in test_data['Y']]

In [10]:
#BERT imports
from transformers import BertTokenizer, BertModel
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [30]:
#BertModel to encode question (X)
bert_q=BertModel.from_pretrained("bert-base-uncased")
for param in bert_q.parameters():
    param.requires_grad = True


Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [11]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0)

'Tesla T4'

In [13]:
new_z1=torch.load('/content/drive/MyDrive/encoded1_doc.pt')

In [14]:
new_z1.shape

torch.Size([7200, 768])

In [15]:
new_z2=torch.load('/content/drive/MyDrive/encoded2_doc.pt')

In [ ]:
type(new_z1)

torch.Tensor

In [16]:
#encoded documents are needed to calculate the loss
train_encodedz=torch.cat((new_z1,new_z2),dim=0)

In [17]:
test_encodedz=torch.load('/content/drive/MyDrive/encoded3_doc.pt')

In [18]:
tokenized_train_x=tokenizer([t for t in train_data['X']],truncation=True,padding=True, return_tensors='pt')
tokenized_test_x=tokenizer([t for t in test_data['X']], truncation=True, padding=True,return_tensors='pt')

In [ ]:
type(tokenized_train_x)

transformers.tokenization_utils_base.BatchEncoding

In [19]:
len(tokenized_train_x['input_ids'])

14011

In [20]:
train_dataset_x=TensorDataset(tokenized_train_x['input_ids'],tokenized_train_x['attention_mask'],tokenized_train_x['token_type_ids'])

In [21]:
test_dataset_x=TensorDataset(tokenized_test_x['input_ids'],tokenized_test_x['attention_mask'],tokenized_test_x['token_type_ids'])

In [22]:
x1_sampler=SequentialSampler(train_dataset_x)
x2_sampler=SequentialSampler(test_dataset_x)
x_train_dataloader=DataLoader(train_dataset_x, sampler=x1_sampler, batch_size=batch_size, shuffle=False)
x_test_dataloader=DataLoader(test_dataset_x, sampler=x2_sampler, batch_size=batch_size, shuffle=False)

In [23]:
encoded_trainset_z=TensorDataset(train_encodedz)
encoded_testset_z=TensorDataset(test_encodedz)

In [24]:
len(encoded_trainset_z)

14000

In [25]:
ez_sampler=SequentialSampler(encoded_trainset_z)
ez_train_dataloader=DataLoader(encoded_trainset_z, sampler=ez_sampler, batch_size=batch_size, shuffle=False)

In [26]:
ezt_sampler=SequentialSampler(encoded_testset_z)
ez_test_dataloader=DataLoader(encoded_testset_z, sampler=ezt_sampler, batch_size=batch_size, shuffle=False)

In [ ]:
len(train_dataset_x)

14011

In [27]:
len(x_train_dataloader), len(ez_train_dataloader)

(1752, 1750)

## question encoder


In [28]:
from transformers import AdamW

In [31]:
optimizer=AdamW(bert_q.parameters(),lr=5e-5)

In [32]:
torch.cuda.empty_cache()

In [ ]:
import torch
device=torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [33]:
bert_q.embeddings.requires_grad = True

In [34]:
from tqdm import tqdm, trange

In [35]:
bert_q.cuda()

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

In [36]:
#loss_q calculates the loss by taking the negative log of the (dot product)^2  of the encoded query and the corresponding document encoding.
# Dot product is squared to avoid taking the log of the negative values.

def loss_q(output,label):
  loss=0
  for i in range(output.shape[0]):
    norm_o= torch.norm(output[i])*torch.norm(label[i])
    dot_o = torch.dot(output[i],label[i])
    dot_o = (dot_o/norm_o)*(dot_o/norm_o)
    loss+= -torch.log(dot_o)
  return loss 


In [46]:
len(x_train_dataloader)

1752

In [51]:
def train_model(bert_q, ez_train_dataloader,x_train_dataloader):
    
      bert_q.train()
    
      epoch_loss = 0
      z_iterator=tqdm(ez_train_dataloader,desc="train_Iteration") #to pick up corresponding document encoded for each query      
      epoch_iterator = tqdm(x_train_dataloader, desc="train_Iteration")
      len(epoch_iterator)
      for step, batch in enumerate(epoch_iterator):
        batch = tuple(t.to(device) for t in batch)
        print(batch)
        inputs = {'input_ids':       batch[0],
                'attention_mask':  batch[1], 
                'token_type_ids':  batch[2]}

        outputs = bert_q(**inputs)
        last_hidden_state=outputs[0]
        encoded_q=last_hidden_state[:,0,:]
        encoded_zq = torch.stack(next(iter(z_iterator))).squeeze()
        loss=loss_q(encoded_q,encoded_zq)
        print(loss)
        epoch_loss=epoch_loss+loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        
        return epoch_loss / len(x_train_dataloader)

In [62]:
def test_model():

  z_iterator=tqdm(ez_test_dataloader,desc="test_Iteration")
  with torch.no_grad():
      epoch_iterator = tqdm(x_test_dataloader, desc="test_Iteration")
      epoch_loss=0
      for step, batch in enumerate(epoch_iterator):
       
        batch = tuple(t.to(device) for t in batch)
        inputs = {'input_ids':       batch[0],
                'attention_mask':  batch[1], 
                'token_type_ids':  batch[2]}
        
        outputs = bert_q(**inputs)
        last_hidden_state=outputs[0]
        encoded_q=last_hidden_state[:,0,:]
        encoded_zq = torch.stack(next(iter(z_iterator))).squeeze()
        loss=loss_q(encoded_q,encoded_zq)
        
        epoch_loss+=loss
  return epoch_loss 
    

In [56]:
import math
from tqdm.auto import tqdm, trange
num_epochs = 3

best_test_loss=float('inf')
bert_q.train()
x_iterator = trange(num_epochs,desc="epochs")
for _ in x_iterator:
      epoch_loss = 0
      z_iterator=tqdm(ez_train_dataloader,desc="train_Iteration") #to pick up corresponding document encoded for each query      
      epoch_iterator = tqdm(x_train_dataloader, desc="train_Iteration")
      len(epoch_iterator)
      for step, batch in enumerate(epoch_iterator):
        batch = tuple(t.to(device) for t in batch)
        #print(batch)
        inputs = {'input_ids':       batch[0],
                'attention_mask':  batch[1], 
                'token_type_ids':  batch[2]}

        outputs = bert_q(**inputs)
        last_hidden_state=outputs[0]
        encoded_q=last_hidden_state[:,0,:]
        encoded_zq = torch.stack(next(iter(z_iterator))).squeeze()
        loss=loss_q(encoded_q,encoded_zq)
        #print(loss)
        epoch_loss=epoch_loss+loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
      print(epoch_loss.item())

  

epochs:   0%|          | 0/3 [00:00<?, ?it/s]

train_Iteration:   0%|          | 0/1750 [00:00<?, ?it/s]

train_Iteration:   0%|          | 0/1752 [00:00<?, ?it/s]

3626.052734375


train_Iteration:   0%|          | 0/1750 [00:00<?, ?it/s]

train_Iteration:   0%|          | 0/1752 [00:00<?, ?it/s]

3471.078857421875


train_Iteration:   0%|          | 0/1750 [00:00<?, ?it/s]

train_Iteration:   0%|          | 0/1752 [00:00<?, ?it/s]

3384.1494140625


In [57]:
torch.save(bert_q.state_dict(), '/content/drive/MyDrive/best_bert_q.pt')

In [58]:
num_epochs = 10

best_test_loss=float('inf')
bert_q.train()
x_iterator = trange(num_epochs,desc="epochs")
for _ in x_iterator:
      epoch_loss = 0
      z_iterator=tqdm(ez_train_dataloader,desc="train_Iteration") #to pick up corresponding document encoded for each query      
      epoch_iterator = tqdm(x_train_dataloader, desc="train_Iteration")
      len(epoch_iterator)
      for step, batch in enumerate(epoch_iterator):
        batch = tuple(t.to(device) for t in batch)
        #print(batch)
        inputs = {'input_ids':       batch[0],
                'attention_mask':  batch[1], 
                'token_type_ids':  batch[2]}

        outputs = bert_q(**inputs)
        last_hidden_state=outputs[0]
        encoded_q=last_hidden_state[:,0,:]
        encoded_zq = torch.stack(next(iter(z_iterator))).squeeze()
        loss=loss_q(encoded_q,encoded_zq)
        #print(loss)
        epoch_loss=epoch_loss+loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
      print(epoch_loss.item())

epochs:   0%|          | 0/10 [00:00<?, ?it/s]

train_Iteration:   0%|          | 0/1750 [00:00<?, ?it/s]

train_Iteration:   0%|          | 0/1752 [00:00<?, ?it/s]

3167.024658203125


train_Iteration:   0%|          | 0/1750 [00:00<?, ?it/s]

...

train_Iteration:   0%|          | 0/1752 [00:00<?, ?it/s]

1898.0250244140625


train_Iteration:   0%|          | 0/1750 [00:00<?, ?it/s]

train_Iteration:   0%|          | 0/1752 [00:00<?, ?it/s]

1821.3433837890625


In [59]:
torch.save(bert_q.state_dict(), '/content/drive/MyDrive/best_bert_qf.pt')

test the model at the last iteration to see how much is the test loss.

In [63]:
test_loss=test_model()
print(test_loss.item())
  

test_Iteration:   0%|          | 0/583 [00:00<?, ?it/s]

test_Iteration:   0%|          | 0/583 [00:00<?, ?it/s]

1865.3214111328125


Another 10 iterations of question encoder

In [64]:
num_epochs = 10
best_loss_so_far=train_loss.item()
best_test_loss=float('inf')
bert_q.train()
x_iterator = trange(num_epochs,desc="epochs")
for _ in x_iterator:
      epoch_loss = 0
      z_iterator=tqdm(ez_train_dataloader,desc="train_Iteration") #to pick up corresponding document encoded for each query      
      epoch_iterator = tqdm(x_train_dataloader, desc="train_Iteration")
      len(epoch_iterator)
      for step, batch in enumerate(epoch_iterator):
        batch = tuple(t.to(device) for t in batch)
        #print(batch)
        inputs = {'input_ids':       batch[0],
                'attention_mask':  batch[1], 
                'token_type_ids':  batch[2]}

        outputs = bert_q(**inputs)
        last_hidden_state=outputs[0]
        encoded_q=last_hidden_state[:,0,:]
        encoded_zq = torch.stack(next(iter(z_iterator))).squeeze()
        loss=loss_q(encoded_q,encoded_zq)
        #print(loss)
        epoch_loss=epoch_loss+loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
      print(epoch_loss.item())
      if epoch_loss.item() < best_loss_so_far:
        best_loss_so_far=epoch_loss.item()
        torch.save(bert_q.state_dict(), '/content/drive/MyDrive/best_bert_qfa.pt')


epochs:   0%|          | 0/10 [00:00<?, ?it/s]

train_Iteration:   0%|          | 0/1750 [00:00<?, ?it/s]

train_Iteration:   0%|          | 0/1752 [00:00<?, ?it/s]

1729.8040771484375


train_Iteration:   0%|          | 0/1750 [00:00<?, ?it/s]

...

train_Iteration:   0%|          | 0/1752 [00:00<?, ?it/s]

1343.23583984375


train_Iteration:   0%|          | 0/1750 [00:00<?, ?it/s]

train_Iteration:   0%|          | 0/1752 [00:00<?, ?it/s]

1344.8001708984375


In [65]:
torch.save(bert_q.state_dict(), '/content/drive/MyDrive/best_bert_qfa.pt')

After another 10 iteration, check the test loss.

In [66]:
test_loss=test_model()
print(test_loss.item())

test_Iteration:   0%|          | 0/583 [00:00<?, ?it/s]

test_Iteration:   0%|          | 0/583 [00:00<?, ?it/s]

1960.107666015625


Save the best model to be uploaded later for FAISS

In [ ]:
'''
  if test_loss < best_test_loss:
        best_test_loss = test_loss
        

  print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
  print(f'\t Test. Loss: {test_loss:.3f} |  Test. PPL: {math.exp(test_loss):7.3f}')
  '''